# Setup

Make sure this notebook works well in both python 2 and 3, import a few common modules, ensure MatplotLib plots figures inline

In [836]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # for retina screens
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

Configure notebook to display all results in cell

In [837]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Revert to the last line of output only
# InteractiveShell.ast_node_interactivity = "last_expr"

# Load Data

In [838]:
import pandas as pd

houses_train = pd.read_csv('../Data/encoded_houses_train.csv')
houses_test = pd.read_csv('../Data/encoded_houses_test.csv')

In [839]:
print("houses_train dimensions: {}".format(houses_train.shape))
print("houses_test dimensions: {}".format(houses_test.shape))

houses_train dimensions: (1460, 288)
houses_test dimensions: (1459, 288)


In [840]:
pd.set_option("display.max_columns", 400)
houses_train.head(3)

,MSSubClass_lev_x.120,MSSubClass_lev_x.160,MSSubClass_lev_x.190,MSSubClass_lev_x.30,MSSubClass_lev_x.50,MSSubClass_lev_x.60,MSSubClass_lev_x.90,MSSubClass_catP,MSSubClass_catN,MSSubClass_catD,MSZoning_lev_x.FV,MSZoning_lev_x.RL,MSZoning_lev_x.RM,MSZoning_catP,MSZoning_catN,MSZoning_catD,LotFrontage_clean,LotArea_clean,Alley_lev_x.Grvl,Alley_lev_x.NA,Alley_catP,Alley_catN,Alley_catD,LotShape_lev_x.IR1,LotShape_lev_x.IR2,LotShape_lev_x.Reg,LotShape_catP,LotShape_catN,LotShape_catD,LandContour_lev_x.Bnk,LandContour_lev_x.HLS,LandContour_catN,LandContour_catD,LotConfig_lev_x.CulDSac,LotConfig_lev_x.Inside,LotConfig_catP,LotConfig_catN,LandSlope_catP,Neighborhood_lev_x.BrkSide,Neighborhood_lev_x.CollgCr,Neighborhood_lev_x.Crawfor,Neighborhood_lev_x.Edwards,Neighborhood_lev_x.IDOTRR,Neighborhood_lev_x.Mitchel,Neighborhood_lev_x.NAmes,Neighborhood_lev_x.NoRidge,Neighborhood_lev_x.NridgHt,Neighborhood_lev_x.OldTown,Neighborhood_lev_x.Sawyer,Neighborhood_lev_x.Somerst,Neighborhood_lev_x.Timber,Neighborhood_catP,Neighborhood_catN,Neighborhood_catD,Condition1_lev_x.Artery,Condition1_lev_x.Feedr,Condition1_lev_x.Norm,Condition1_catP,Condition1_catN,Condition1_catD,BldgType_lev_x.1Fam,BldgType_lev_x.2fmCon,BldgType_lev_x.Duplex,BldgType_lev_x.Twnhs,BldgType_catP,BldgType_catN,BldgType_catD,HouseStyle_lev_x.1.5Fin,HouseStyle_lev_x.1Story,HouseStyle_lev_x.2Story,HouseStyle_lev_x.SFoyer,HouseStyle_catP,HouseStyle_catN,HouseStyle_catD,OverallQual_clean,OverallCond_clean,YearBuilt_clean,YearRemodAdd_clean,RoofStyle_lev_x.Gable,RoofStyle_lev_x.Hip,RoofStyle_catP,RoofStyle_catN,RoofStyle_catD,RoofMatl_lev_x.CompShg,RoofMatl_catP,RoofMatl_catN,Exterior1st_lev_x.Cement,Exterior1st_lev_x.HdBoard,Exterior1st_lev_x.MetalSd,Exterior1st_lev_x.Shingles,Exterior1st_lev_x.VinylSd,Exterior1st_lev_x.Wd.Sdng,Exterior1st_catP,Exterior1st_catN,Exterior1st_catD,Exterior2nd_lev_x.Cement,Exterior2nd_lev_x.HdBoard,Exterior2nd_lev_x.MetalSd,Exterior2nd_lev_x.Plywood,Exterior2nd_lev_x.VinylSd,Exterior2nd_lev_x.Wd.Sdng,Exterior2nd_catP,Exterior2nd_catN,Exterior2nd_catD,MasVnrType_lev_x.BrkFace,MasVnrType_lev_x.None,MasVnrType_lev_x.Stone,MasVnrType_catP,MasVnrType_catN,MasVnrType_catD,MasVnrArea_clean,ExterQual_lev_x.3,ExterQual_lev_x.4,ExterQual_lev_x.5,ExterQual_catP,ExterQual_catN,ExterQual_catD,ExterCond_lev_x.TA,ExterCond_catP,ExterCond_catN,ExterCond_catD,Foundation_lev_x.BrkTil,Foundation_lev_x.CBlock,Foundation_lev_x.PConc,Foundation_catP,Foundation_catN,Foundation_catD,BsmtQual_lev_x.Ex,BsmtQual_lev_x.Fa,BsmtQual_lev_x.Gd,BsmtQual_lev_x.NA,BsmtQual_lev_x.TA,BsmtQual_catP,BsmtQual_catN,BsmtQual_catD,BsmtCond_lev_x.Fa,BsmtCond_lev_x.Gd,BsmtCond_lev_x.NA,BsmtCond_lev_x.TA,BsmtCond_catP,BsmtCond_catN,BsmtCond_catD,BsmtExposure_lev_x.Av,BsmtExposure_lev_x.Gd,BsmtExposure_lev_x.NA,BsmtExposure_lev_x.No,BsmtExposure_catP,BsmtExposure_catN,BsmtExposure_catD,BsmtFinType1_lev_x.ALQ,BsmtFinType1_lev_x.BLQ,BsmtFinType1_lev_x.GLQ,BsmtFinType1_lev_x.LwQ,BsmtFinType1_lev_x.NA,BsmtFinType1_lev_x.Rec,BsmtFinType1_lev_x.Unf,BsmtFinType1_catP,BsmtFinType1_catN,BsmtFinType1_catD,BsmtFinSF1_clean,BsmtFinType2_lev_x.BLQ,BsmtFinType2_lev_x.NA,BsmtFinType2_lev_x.Unf,BsmtFinType2_catP,BsmtFinType2_catN,BsmtFinType2_catD,BsmtUnfSF_clean,TotalBsmtSF_clean,Heating_lev_x.GasA,Heating_catP,Heating_catN,Heating_catD,HeatingQC_lev_x.2,HeatingQC_lev_x.3,HeatingQC_lev_x.4,HeatingQC_lev_x.5,HeatingQC_catP,HeatingQC_catN,HeatingQC_catD,CentralAir_lev_x.N,CentralAir_lev_x.Y,Electrical_lev_x.FuseA,Electrical_lev_x.FuseFP,Electrical_lev_x.SBrkr,Electrical_catP,Electrical_catN,Electrical_catD,X1stFlrSF_clean,X2ndFlrSF_clean,GrLivArea_clean,BsmtFullBath_clean,FullBath_clean,HalfBath_clean,BedroomAbvGr_clean,KitchenAbvGr_clean,KitchenQual_lev_x.2,KitchenQual_lev_x.3,KitchenQual_lev_x.4,KitchenQual_lev_x.5,KitchenQual_catP,KitchenQual_catN,KitchenQual_catD,TotRmsAbvGrd_clean,Functional_lev_x.Min1,Functional_lev_x.Min2,Functional_lev_x.Typ,Functional_catP,Functional_catN,Fireplaces_clean,FireplaceQu_le

In [841]:
houses_train.info()
houses_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 288 entries, MSSubClass_lev_x.120 to SalePrice
dtypes: float64(288)
memory usage: 3.2 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Columns: 288 entries, MSSubClass_lev_x.120 to SalePrice
dtypes: float64(287), int64(1)
memory usage: 3.2 MB


Delete first column

In [842]:
# houses_train.drop('Unnamed: 0', axis=1, inplace=True, errors='raise')
# houses_test.drop('Unnamed: 0', axis=1, inplace=True, errors='raise')

# Run xgboost

### Create private training & test set

In [843]:
from sklearn.model_selection import train_test_split

seed = 10
test_ratio = 0.2

X = houses_train.loc[:, houses_train.columns != "SalePrice"].values # convert to np.array
y = houses_train.loc[:, houses_train.columns == "SalePrice"].values

# Take log of SalePrice
y = np.log(y + 1).ravel() # convert to 1D array for model fit (xxx, )


In [844]:
X_pr_train, X_pr_test, y_pr_train, y_pr_test = train_test_split(X, y, test_size=test_ratio, random_state=seed)

In [845]:
print(len(X_pr_train), "train +", len(X_pr_test), "test")

1168 train + 292 test


### Fit Model

In [846]:
from xgboost import XGBRegressor

xgb_clf = XGBRegressor(max_depth=3, 
                        learning_rate=0.05, 
                        n_estimators=1000, # Number of boosted trees to fit
                        silent=False, # print messages while running 
                        objective='reg:linear', 
                        booster='gbtree', # Specify which booster to use: gbtree, gblinear or dart
                        #for dart see http://xgboost.readthedocs.io/en/latest/tutorials/dart.html 
                        n_jobs=-1, # Number of parallel threads used to run xgboost. (replaces nthread)
                        gamma=0,  # Minimum loss reduction required to make a further partition on a leaf node of the tree.
                        min_child_weight=1, # Minimum sum of instance weight(hessian) needed in a child
                        max_delta_step=0, # Maximum delta step we allow each tree’s weight estimation to be
                        subsample=1, # Subsample ratio of the training instance
                        colsample_bytree=1, # Subsample ratio of columns when constructing each tree
                        colsample_bylevel=1, # Subsample ratio of columns for each split, in each level
                        reg_alpha=0, # L1 regularization term on weights
                        reg_lambda=1, # L2 regularization term on weights
                        scale_pos_weight=1, # Balancing of positive and negative weights
                        base_score=0.5, # The initial prediction score of all instances, global bias
                        random_state=743, 
                        missing=None) # Value in the data which needs to be present as a missing value. If None, defaults to np.nan



In [847]:
xgb_clf.fit(X_pr_train, y_pr_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=-1, nthread=None, objective='reg:linear', random_state=743,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=1)

In [848]:
# make predictions for test data

y_pr_pred = xgb_clf.predict(X_pr_test)

Evaluate predictions

In [849]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_pr_test, y_pr_pred)
rmse = np.sqrt(mse)
rmse


0.12373572496003182

In [850]:
y_pr_test[1:5]
y_pr_pred[1:5]

array([ 12.1428719 ,  11.8277435 ,  12.01067193,  12.64109979])

array([ 12.22154427,  11.80571461,  11.84539032,  12.69584465], dtype=float32)

Save model to file

In [851]:
from sklearn.externals import joblib # More memory efficient than pickle for large numpy arrays

joblib.dump(xgb_clf, './Models/xgboost_model.pkl') 

['./Models/xgboost_model.pkl']

To load a model:

In [852]:
# xgb_clf_loaded = joblib.load('./Models/xgboost_model.pkl') 

### Model Tuning

In [853]:
from sklearn.model_selection import GridSearchCV

xgb_params = {'max_depth': 3,
              'learning_rate': 0.1, 
              'n_estimators': 100, 
              'objective': 'reg:linear'}

# param_grid = {'max_depth': [3], 
#               'learning_rate': [0.05], 
#               'n_estimators': [2000], # Number of boosted trees to fit
#               'objective': ['reg:linear'], 
#               'booster': ['gbtree'], # Specify which booster to use: gbtree, gblinear or dart
#               'n_jobs': [-1], # Number of parallel threads used to run xgboost. (replaces nthread)
#               'gamma': [0],  # Minimum loss reduction required to make a further partition on a leaf node of the tree.
#               'min_child_weight': [1], # Minimum sum of instance weight(hessian) needed in a child
#               'max_delta_step': [0], # Maximum delta step we allow each tree’s weight estimation to be
#               'subsample': [1], # Subsample ratio of the training instance
#               'colsample_bytree': [0.5], # Subsample ratio of columns when constructing each tree
#               'colsample_bylevel': [0.4], # Subsample ratio of columns for each split, in each level
#               'reg_alpha': [0], # L1 regularization term on weights
#               'reg_lambda': [1], # L2 regularization term on weights
#               'scale_pos_weight': [1], # Balancing of positive and negative weights
#               'base_score': [0.5], # The initial prediction score of all instances, global bias
#               'silent': [True],
#               'random_state': [10]}

param_grid = {'max_depth': [3], 
              'learning_rate': [0.05, 0.07, 0.09], 
              'n_estimators': [1500, 1800, 2000], # Number of boosted trees to fit
              'objective': ['reg:linear'], 
              'booster': ['gbtree'], # Specify which booster to use: gbtree, gblinear or dart
              'n_jobs': [-1], # Number of parallel threads used to run xgboost. (replaces nthread)
              'gamma': [0, 0.5],  # Minimum loss reduction required to make a further partition on a leaf node of the tree.
              'min_child_weight': [1], # Minimum sum of instance weight(hessian) needed in a child
              'max_delta_step': [0], # Maximum delta step we allow each tree’s weight estimation to be
              'subsample': [1], # Subsample ratio of the training instance
              'colsample_bytree': [0.5, 0.6, 0.7], # Subsample ratio of columns when constructing each tree
              'colsample_bylevel': [0.2, 0.3, 0.4], # Subsample ratio of columns for each split, in each level
              'reg_alpha': [0], # L1 regularization term on weights
              'reg_lambda': [1], # L2 regularization term on weights
              'scale_pos_weight': [1], # Balancing of positive and negative weights
              'base_score': [0.5], # The initial prediction score of all instances, global bias
              'silent': [True],
              'random_state': [10]}

optimized_xgb_clf = GridSearchCV(XGBRegressor(**xgb_params), # scikit-learn estimator interface 
                                 param_grid = param_grid, # Dictionary with parameters names (string) as keys
                                 scoring="neg_mean_squared_error", # controls what metric they apply to the estimators evaluated
                                 n_jobs=-1, # If True, the data is assumed to be identically distributed across the folds,
                                 iid=True, 
                                 refit=True, # Refit an estimator using the best found parameters (best_estimator_)
                                 cv=3, #integer, to specify the number of folds in a (Stratified)KFold. None -> default 3-fold cross validation
                                 verbose=10, # the higher, the more messages
                                 pre_dispatch="2*n_jobs", # number of jobs that get dispatched during parallel execution
                                 error_score="raise", 
                                 return_train_score=False) #If False, the cv_results_ attribute will not include training scores

Inspect the grid

In [ ]:
optimized_xgb_clf

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3], 'learning_rate': [0.05, 0.07, 0.09], 'n_estimators': [1500, 1800, 2000], 'objective': ['reg:linear'], 'booster': ['gbtree'], 'n_jobs': [-1], 'gamma': [0, 0.5], 'min_child_weight': [1], 'max_delta_step': [0], 'subsample': [1], 'colsample_bytree': [0.5, 0.6, 0.7], 'colsample_bylevel': [0.2, 0.3, 0.4], 'reg_alpha': [0], 'reg_lambda': [1], 'scale_pos_weight': [1], 'base_score': [0.5], 'silent': [True], 'random_state': [10]},
       pre_dispatch='2*n_jobs', refit=True, return_tr

Run grid tuning

In [ ]:
optimized_xgb_clf.fit(X_pr_train, y_pr_train)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
[CV] base_score=0.5, booster=gbtree, colsample_bylevel=0.2, colsample_bytree=0.5, gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1500, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1 
[CV] base_score=0.5, booster=gbtree, colsample_bylevel=0.2, colsample_bytree=0.5, gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1500, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1 
[CV] base_score=0.5, booster=gbtree, colsample_bylevel=0.2, colsample_bytree=0.5, gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1800, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1 
[CV] base_score=0.5, boos

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    7.9s


[CV]  base_score=0.5, booster=gbtree, colsample_bylevel=0.2, colsample_bytree=0.5, gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=2000, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1, score=-0.015882879480013438, total=   4.6s
[CV] base_score=0.5, booster=gbtree, colsample_bylevel=0.2, colsample_bytree=0.5, gamma=0, learning_rate=0.07, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1500, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1 
[CV]  base_score=0.5, booster=gbtree, colsample_bylevel=0.2, colsample_bytree=0.5, gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=2000, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1, score=-0.01426418179182491, total=   4.5s
[

[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   12.1s


[CV]  base_score=0.5, booster=gbtree, colsample_bylevel=0.2, colsample_bytree=0.5, gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=2000, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1, score=-0.015397253074609015, total=   4.7s
[CV] base_score=0.5, booster=gbtree, colsample_bylevel=0.2, colsample_bytree=0.5, gamma=0, learning_rate=0.07, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1800, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1 
[CV]  base_score=0.5, booster=gbtree, colsample_bylevel=0.2, colsample_bytree=0.5, gamma=0, learning_rate=0.07, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1500, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1, score=-0.014896910615445118, total=   3.5s


...

[Parallel(n_jobs=-1)]: Done 234 tasks      | elapsed:  5.6min


[CV]  base_score=0.5, booster=gbtree, colsample_bylevel=0.3, colsample_bytree=0.6, gamma=0, learning_rate=0.09, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1500, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1, score=-0.01607022490137085, total=   5.9s
[CV] base_score=0.5, booster=gbtree, colsample_bylevel=0.3, colsample_bytree=0.6, gamma=0, learning_rate=0.09, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1800, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1 
[CV]  base_score=0.5, booster=gbtree, colsample_bylevel=0.3, colsample_bytree=0.6, gamma=0, learning_rate=0.09, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1500, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1, score=-0.013486314897466848, total=   5.9s
[

[CV]  base_score=0.5, booster=gbtree, colsample_bylevel=0.3, colsample_bytree=0.6, gamma=0.5, learning_rate=0.05, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1800, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1, score=-0.020768675540582395, total=   7.0s
[CV] base_score=0.5, booster=gbtree, colsample_bylevel=0.3, colsample_bytree=0.6, gamma=0.5, learning_rate=0.05, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=2000, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1 
[CV]  base_score=0.5, booster=gbtree, colsample_bylevel=0.3, colsample_bytree=0.6, gamma=0.5, learning_rate=0.05, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1800, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1, score=-0.019890639713141055, total=  

[Parallel(n_jobs=-1)]: Done 257 tasks      | elapsed:  6.2min


[CV]  base_score=0.5, booster=gbtree, colsample_bylevel=0.3, colsample_bytree=0.6, gamma=0.5, learning_rate=0.07, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1800, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1, score=-0.020099730604727317, total=   7.2s
[CV] base_score=0.5, booster=gbtree, colsample_bylevel=0.3, colsample_bytree=0.6, gamma=0.5, learning_rate=0.09, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1500, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1 
[CV]  base_score=0.5, booster=gbtree, colsample_bylevel=0.3, colsample_bytree=0.6, gamma=0.5, learning_rate=0.07, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=2000, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1, score=-0.026082645420525526, total=  

The best combination of parameters is:

In [ ]:
optimized_xgb_clf.best_params_

In [ ]:
## The best score is
optimized_xgb_clf.best_score_ # that's the training score so not meaningful

In [ ]:
# optimized_xgb_clf.cv_results_

In [ ]:
# make predictions for test data

y_pr_pred = optimized_xgb_clf.predict(X_pr_test)

Evaluate predictions

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_pr_test, y_pr_pred)
rmse = np.sqrt(mse)
rmse


In [ ]:
y_pr_test[1:5]
y_pr_pred[1:5]